# Visualize Point Clouds

This visualization assumes you've already generated the original point clouds and images in the correct folders. Be sure to run `pcs_preprocess.py` with the `-i -o` options first to create the required datasets before you run this script.

## Initialization

Change `ROOT_DIR` to dataset path.

In [ ]:
try:
    import rerun as rr
    import matplotlib.pyplot as plt

except ImportError:
  print("The modules are missing. Installing now via pip...")
  # These versions of numpy and pandas-gbq are compatible with the rest of the colab environment
  %pip install rerun-sdk rerun-notebook matplotlib
  print('Installation completed. The runtime needs to be restarted. Stopping now.')
  import os
  os.kill(os.getpid(), 9)

import os
import numpy as np
from tqdm import tqdm
import re


ROOT_DIR = "/datasets/snail-radar/81r/20240116_2_preprocessed"

## Visualize Point Clouds

Change `start_timestamp` and `end_timestamp` to specify time period to visalize.

In [ ]:
# Parameters for processed point clouds and visualization.
min_speed_, max_speed_ = 0.4, 1   # Processed point cloud speed range
min_power, max_power = 0, 30
start_timestamp, end_timestamp = 1705391169, 1705391175

# Initialize visualization
rr.init("viz_pcs")
rr.notebook_show(width=1200, height=800)

# Utility functions
def reverse_timestamp(timestamp_str):
    """
    Convert a timestamp string to a float by inserting a decimal point six characters from the right.
    """
    timestamp_str = timestamp_str.zfill(7)
    return float(timestamp_str[:-6] + "." + timestamp_str[-6:])

def extract_timestamp(filename):
    """
    Extract the numeric timestamp from the filename using a regex.
    Expected to work for both processed and original point cloud files.
    """
    match = re.search(r'(\d+)(?=_org\.npy|\.npy)', filename)
    return int(match.group(1)) if match else None

def normalize_and_color_speeds(speeds, cmap):
    """
    Normalize positive and negative speed components separately
    and then remap the normalized values [-1, 1] to [0, 1] for applying the colormap.
    """
    positive = speeds[speeds > 0]
    negative = speeds[speeds < 0]
    max_pos = positive.max() if positive.size > 0 else 1
    min_neg = negative.min() if negative.size > 0 else -1
    normalized = np.zeros_like(speeds)
    if positive.size > 0:
        normalized[speeds > 0] = positive / max_pos
    if negative.size > 0:
        normalized[speeds < 0] = negative / abs(min_neg)
    return cmap((normalized + 1) / 2)[:, :3]

def set_colors_to_black(speeds):
    """
    For processed point clouds we simply use black for the speed colors.
    """
    return np.zeros((speeds.shape[0], 3))

# Define directories
points_dir = os.path.join(ROOT_DIR, "pointclouds")
images_dir = os.path.join(ROOT_DIR, "images")


orig_files = {f for f in os.listdir(points_dir) if f.endswith('_org.npy')}

image_files = {}
for f in os.listdir(images_dir):
    m = re.match(r'(\d+)', f)
    if m:
        ts_key = int(m.group(1))
        image_files[ts_key] = os.path.join(images_dir, f)

all_files = [f for f in os.listdir(points_dir) if f.endswith('.npy') and '_' not in f]

start_ts_int = int("{:.6f}".format(start_timestamp).replace(".", ""))
end_ts_int = int("{:.6f}".format(end_timestamp).replace(".", ""))

processed_files = []
for f in all_files:
    ts = extract_timestamp(f)
    if ts is not None and start_ts_int <= ts <= end_ts_int:
        processed_files.append((f, ts))

processed_files = sorted(processed_files, key=lambda x: x[1])

unique_ts = sorted({ts for _, ts in processed_files})
timestamp_to_step = {ts: i for i, ts in enumerate(unique_ts)}

speed_cmap = plt.get_cmap('coolwarm')
power_cmap = plt.get_cmap('Blues')

for filename, ts in tqdm(processed_files, desc="Logging Processed Point Clouds In Order"):
    step = timestamp_to_step[ts]
    timestamp_float = reverse_timestamp(str(ts))
    proc_filepath = os.path.join(points_dir, filename)
    
    # Load the processed point cloud using memory-mapping for efficiency.
    processed_pc = np.load(proc_filepath, mmap_mode='r')
    positions = processed_pc[:, :3]
    speeds = processed_pc[:, 3]
    powers = processed_pc[:, 4]
    
    speeds = np.clip(speeds, min_speed_, max_speed_)
    normalized_speeds = (speeds - min_speed_) / (max_speed_ - min_speed_)
    colors_speed = set_colors_to_black(normalized_speeds)
    
    normalized_powers = (powers - min_power) / (max_power - min_power)
    colors_power = power_cmap(normalized_powers)[:, :3]
    
    rr.set_time_sequence("step", step)
    rr.log(
        "processed_pc",
        rr.Points3D(positions, radii=0.6, colors=colors_speed,
                    labels=f"{timestamp_float:.3f}")
    )
    
    orig_filename = filename.replace(".npy", "_org.npy")
    if orig_filename in orig_files:
        orig_filepath = os.path.join(points_dir, orig_filename)
        orig_pc = np.load(orig_filepath, mmap_mode='r')
        orig_positions = orig_pc[:, :3]
        orig_speeds = orig_pc[:, 3]
        colors_orig = normalize_and_color_speeds(orig_speeds, speed_cmap)
        rr.log(
            "original_pc",
            rr.Points3D(orig_positions, radii=0.6, colors=colors_orig,
                        labels=f"{timestamp_float:.3f}")
        )
    
    if ts in image_files:
        image = plt.imread(image_files[ts])
        rr.log("image", rr.Image(image))